In [15]:
pip install pandas nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Load Dataset
df = pd.read_csv(r"C:\Users\rohit\Downloads\micro intern\archive (5).zip")  # Change the filename if needed

# Display the first 5 rows
print(df.head())

# Check available columns
print(df.columns)

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [4]:
# Keep only 'Score' and 'Text' columns
df = df[['Score', 'Text']]

# Remove missing values
df = df.dropna()

# Show dataset info
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Score   568454 non-null  int64 
 1   Text    568454 non-null  object
dtypes: int64(1), object(1)
memory usage: 8.7+ MB
None


In [6]:
def convert_score_to_sentiment(score):
    if score >= 4:
        return "Positive"
    elif score == 3:
        return "Neutral"
    else:
        return "Negative"

# Apply conversion
df["Sentiment"] = df["Score"].apply(convert_score_to_sentiment)

# Remove original 'Score' column
df = df.drop(columns=["Score"])

# Show sentiment distribution
print(df["Sentiment"].value_counts())


Sentiment
Positive    443777
Negative     82037
Neutral      42640
Name: count, dtype: int64


In [8]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary data for text processing
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rohit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rohit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rohit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W+', ' ', text)  # Remove special characters
    tokens = word_tokenize(text)  # Tokenization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Stopword removal & lemmatization
    return " ".join(tokens)

# Apply text cleaning
df["Cleaned_Text"] = df["Text"].astype(str).apply(preprocess_text)

# Show cleaned text samples
print(df[["Text", "Cleaned_Text", "Sentiment"]].head())


                                                Text  \
0  I have bought several of the Vitality canned d...   
1  Product arrived labeled as Jumbo Salted Peanut...   
2  This is a confection that has been around a fe...   
3  If you are looking for the secret ingredient i...   
4  Great taffy at a great price.  There was a wid...   

                                        Cleaned_Text Sentiment  
0  bought several vitality canned dog food produc...  Positive  
1  product arrived labeled jumbo salted peanut pe...  Negative  
2  confection around century light pillowy citrus...  Positive  
3  looking secret ingredient robitussin believe f...  Negative  
4  great taffy great price wide assortment yummy ...  Positive  


In [12]:
df.to_csv("Cleaned_Reviews.csv", index=False)
print("\n✅ Preprocessing Complete! Cleaned dataset saved as 'Cleaned_Reviews.csv'")



✅ Preprocessing Complete! Cleaned dataset saved as 'Cleaned_Reviews.csv'


In [14]:
pip install scikit-learn numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd

# Load the cleaned dataset
df = pd.read_csv("Cleaned_Reviews.csv")

# Show first 5 rows
print(df.head())

# Check data distribution
print(df["Sentiment"].value_counts())


                                                Text Sentiment  \
0  I have bought several of the Vitality canned d...  Positive   
1  Product arrived labeled as Jumbo Salted Peanut...  Negative   
2  This is a confection that has been around a fe...  Positive   
3  If you are looking for the secret ingredient i...  Negative   
4  Great taffy at a great price.  There was a wid...  Positive   

                                        Cleaned_Text  
0  bought several vitality canned dog food produc...  
1  product arrived labeled jumbo salted peanut pe...  
2  confection around century light pillowy citrus...  
3  looking secret ingredient robitussin believe f...  
4  great taffy great price wide assortment yummy ...  
Sentiment
Positive    443777
Negative     82037
Neutral      42640
Name: count, dtype: int64


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text into numerical features using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Use 5000 most important words
X = vectorizer.fit_transform(df["Cleaned_Text"])

# Convert labels to numerical values (0 = Negative, 1 = Neutral, 2 = Positive)
sentiment_mapping = {"Negative": 0, "Neutral": 1, "Positive": 2}
y = df["Sentiment"].map(sentiment_mapping)

# Show the shape of the transformed data
print("Feature Matrix Shape:", X.shape)


Feature Matrix Shape: (568454, 5000)


In [20]:
from sklearn.model_selection import train_test_split

# Split into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Show dataset sizes
print("Training Set Size:", X_train.shape[0])
print("Testing Set Size:", X_test.shape[0])


Training Set Size: 454763
Testing Set Size: 113691


In [24]:
from sklearn.linear_model import LogisticRegression

# Initialize and train the model
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)


In [26]:
from sklearn.metrics import accuracy_score, classification_report

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

# Show detailed classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Model Accuracy: 0.8664186259246555

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.67      0.70     16181
           1       0.51      0.17      0.26      8485
           2       0.90      0.97      0.93     89025

    accuracy                           0.87    113691
   macro avg       0.71      0.60      0.63    113691
weighted avg       0.85      0.87      0.85    113691



In [28]:
# Define custom reviews
custom_reviews = [
    "This product is amazing! I love it.",
    "It's okay, not the best but not the worst.",
    "Horrible experience, I would never buy this again!"
]

# Convert text to TF-IDF features
custom_vectors = vectorizer.transform(custom_reviews)

# Predict sentiment
predictions = model.predict(custom_vectors)

# Convert back to text labels
label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}
predicted_sentiments = [label_mapping[p] for p in predictions]

# Display results
for review, sentiment in zip(custom_reviews, predicted_sentiments):
    print(f"Review: {review} → Sentiment: {sentiment}")


Review: This product is amazing! I love it. → Sentiment: Positive
Review: It's okay, not the best but not the worst. → Sentiment: Negative
Review: Horrible experience, I would never buy this again! → Sentiment: Negative


In [30]:
pip install joblib


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [32]:
import joblib

# Save the trained model
joblib.dump(model, "sentiment_model.pkl")

# Save the TF-IDF vectorizer 
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

print("\n✅ Model and Vectorizer Saved Successfully!")



✅ Model and Vectorizer Saved Successfully!
